In [1]:
import numpy as np

In [5]:
np.arange(24).reshape(2, 3, 4).T.shape

(4, 3, 2)

In [16]:
x = np.arange(10).reshape(1, 2, 5)
y = np.ones(2)
x, y

(array([[[0, 1, 2, 3, 4],
         [5, 6, 7, 8, 9]]]),
 array([1., 1.]))

In [15]:
np.einsum("i n d, n -> d", x, y)

array([ 5.,  7.,  9., 11., 13.])